## 작업 1유형

* 데이터 출처 : https://archive.ics.uci.edu/ml/datasets/Bank+Marketing (후처리 작업)
* 데이터 설명 : 은행의 전화 마케팅에 대해 고객의 반응 여부
* dataurl : https://raw.githubusercontent.com/Datamanim/datarepo/main/bank/train.csv

In [1]:
import pandas as pd

dataurl = 'https://raw.githubusercontent.com/Datamanim/datarepo/main/bank/train.csv'
bank = pd.read_csv(dataurl)
bank.head()

,ID,age,job,marital,education,default,balance,housing,loan,contact,day,month,campaign,pdays,previous,poutcome,y
0,13829,29,technician,single,tertiary,no,18254,no,no,cellular,11,may,2,-1,0,unknown,no
1,22677,26,services,single,secondary,no,512,yes,yes,unknown,5,jun,3,-1,0,unknown,no
2,10541,30,management,single,secondary,no,135,no,no,cellular,14,aug,2,-1,0,unknown,no
3,13689,41,technician,married,unknown,no,30,yes,no,cellular,10,jul,1,-1,0,unknown,no
4,11304,27,admin.,single,secondary,no,321,no,yes,unknown,2,sep,1,-1,0,unknown,no


#### Q1. 마케팅 응답 고객들의 나이를 10살 단위로 변환 했을 때, 가장 많은 인원을 가진 나이대는? (0~9 : 0 , 10~19 : 10)

In [4]:
bank['ageCG'] = (bank.age//10)*10
bank['ageCG'].value_counts().index[0]

30

#### Q2. 마케팅 응답 고객들의 나이를 10살 단위로 변환 했을 때, 가장 많은 나이대 구간의 인원은 몇명인가?

In [10]:
bank['ageCG'].value_counts().values[0]

5056

#### Q3. 나이가 25살 이상 29살 미만인 응답 고객들중 housing컬럼의 값이 yes인 고객의 수는?

In [13]:
len(bank[(bank.age >=25) & (bank.age<29) & (bank.housing =='yes')])

504

#### Q4. numeric한 값을 가지지 않은 컬럼들중 unique한 값을 가장 많이 가지는 컬럼은? ★

In [15]:
bank.dtypes

ID            int64
age           int64
job          object
marital      object
education    object
default      object
balance       int64
housing      object
loan         object
contact      object
day           int64
month        object
campaign      int64
pdays         int64
previous      int64
poutcome     object
y            object
ageCG         int64
dtype: object

In [40]:
lst = []
for val in bank.select_dtypes(exclude = "int"):
    t = bank[val]
    lst.append([val, t.nunique()])

pd.DataFrame(lst).sort_values(1, ascending = False).values[0][0]

'job'

#### Q5. balance 컬럼값들의 평균값 이상을 가지는 데이터를 ID값을 기준으로 내림차순 정렬했을때 상위 100개 데이터의 balance값의 평균은?

In [43]:
bank[bank.balance >= bank.balance.mean()].sort_values(by = 'ID', ascending = False).head(100)['balance'].mean()

3473.73

#### Q6. 가장 많은 광고를 집행했던 날짜는 언제인가? (데이터 그대로 일(숫자),달(영문)으로 표기)

In [46]:
bank.groupby(['day', 'month']).size().sort_values(ascending = False).index[0]

(15, 'may')

#### Q7. 데이터의 job이 unknown 상태인 고객들의 age 컬럼 값의 정규성을 검정하고자 한다. 샤피로 검정의 p-value값을 구하여라 ★

In [48]:
from scipy.stats import shapiro

shapiro(bank[bank.job == 'unknown'].age)[1]

0.1961131989955902

#### Q8. age와 balance의 상관계수를 구하여라

In [51]:
bank[['age', 'balance']].corr().iloc[0,1]

0.10198734763981462

#### Q9. y 변수와 education 변수는 독립인지 카이제곱검정을 통해 확인하려한다. p-value값을 출력하라 ★

In [55]:
v = pd.crosstab(bank.y, bank.education)

from scipy.stats import chi2_contingency
chic2, p, dof, expected = chi2_contingency(v)
p

7.901201277473551e-29

#### Q10. 각 job에 따라 divorced/married 인원의 비율을 확인 했을 때 그 값이 가장 높은 값은? ★

In [63]:
tmp = bank.groupby(['job', 'marital']).size().reset_index()
pivotdf = tmp.pivot_table(index = 'job', columns='marital')[0]
pivotdf = pivotdf.fillna(0)
pivotdf['ratio'] = pivotdf['divorced'] / pivotdf['married']
pivotdf.sort_values('ratio').ratio.values[-1]

0.2831050228310502

## 작업 2유형

* 데이터 출처 : https://archive.ics.uci.edu/ml/datasets/Bank+Marketing (후처리 작업)
* 데이터 설명 : 은행의 전화 마케팅에 대해 고객의 반응 여부
* train : https://raw.githubusercontent.com/Datamanim/datarepo/main/bank/train.csv
* test : https://raw.githubusercontent.com/Datamanim/datarepo/main/bank/test.csv
* submission : https://raw.githubusercontent.com/Datamanim/datarepo/main/bank/submission.csv

In [124]:
train = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/bank/train.csv')
test = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/bank/test.csv')
submission = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/bank/submission.csv')
train.head()

,ID,age,job,marital,education,default,balance,housing,loan,contact,day,month,campaign,pdays,previous,poutcome,y
0,13829,29,technician,single,tertiary,no,18254,no,no,cellular,11,may,2,-1,0,unknown,no
1,22677,26,services,single,secondary,no,512,yes,yes,unknown,5,jun,3,-1,0,unknown,no
2,10541,30,management,single,secondary,no,135,no,no,cellular,14,aug,2,-1,0,unknown,no
3,13689,41,technician,married,unknown,no,30,yes,no,cellular,10,jul,1,-1,0,unknown,no
4,11304,27,admin.,single,secondary,no,321,no,yes,unknown,2,sep,1,-1,0,unknown,no


In [125]:
test.head()

,ID,age,job,marital,education,default,balance,housing,loan,contact,day,month,campaign,pdays,previous,poutcome
0,53608,32,management,single,tertiary,no,12569,no,no,cellular,1,jul,2,295,2,success
1,51055,25,services,single,secondary,no,801,no,no,cellular,5,jun,2,-1,0,unknown
2,52573,46,blue-collar,married,secondary,no,1728,yes,no,unknown,26,may,2,-1,0,unknown
3,50458,39,management,divorced,secondary,no,51,no,no,unknown,17,jun,2,-1,0,unknown
4,52272,31,services,single,tertiary,no,1626,no,no,unknown,31,jul,1,-1,0,unknown


In [126]:
submission.head()

,ID,predict
0,53608,0.0
1,51055,0.0
2,52573,0.0
3,50458,0.0
4,52272,0.0


#### 모델링 및 submission파일 생성까지
##### 데이터 확인

In [127]:
train.isnull().sum()

ID           0
age          0
job          0
marital      0
education    0
default      0
balance      0
housing      0
loan         0
contact      0
day          0
month        0
campaign     0
pdays        0
previous     0
poutcome     0
y            0
dtype: int64

In [128]:
train.dtypes

ID            int64
age           int64
job          object
marital      object
education    object
default      object
balance       int64
housing      object
loan         object
contact      object
day           int64
month        object
campaign      int64
pdays         int64
previous      int64
poutcome     object
y            object
dtype: object

##### 데이터 전처리

In [129]:
x = train.drop(['ID', 'y'], axis=1)
test = test.drop('ID', axis = 1)
y = train['y']

In [130]:
train = pd.get_dummies(x)
test = pd.get_dummies(test)

In [131]:
from sklearn.model_selection import train_test_split

train_x, val_x, train_y, val_y = train_test_split(train, y)

In [132]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier()
rf.fit(train_x, train_y)

RandomForestClassifier()

In [147]:
val_sc = rf.predict_proba(val_x)[:,1]

In [148]:
from sklearn.metrics import roc_auc_score

roc_auc_score(val_y, val_sc)

0.7776640732430249

In [149]:
test_proba = rf.predict_proba(test)[:,1]

In [152]:
submission['predict'] = test_proba
submission.head()

,ID,predict
0,53608,0.81
1,51055,0.80
2,52573,0.03
3,50458,0.25
4,52272,0.28


In [153]:
submission.to_csv('1회차.csv', index=False)